In [ ]:
!pip install -q streamlit

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.125.209.87


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session 183cc90c2d23244


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.209.87:8501

npx: installed 22 in 2.746s
your url is: https://green-tips-care.loca.lt
  Stopping...
^C


In [ ]:
import pickle
!pip install gdown
import gdown

In [ ]:
pip install efficientnet-pytorch

In [ ]:
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

class ModifiedEfficientNet(nn.Module):
    def __init__(self, num_classes, model_name='efficientnet-b0', dropout_rate=0.2):
        super(ModifiedEfficientNet, self).__init__()

        # Load the pre-trained EfficientNet model
        self.efficientnet = EfficientNet.from_pretrained(model_name)

        # Additional convolutional layer with dropout
        self.additional_conv = nn.Sequential(
            nn.Conv2d(1280, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
        )

        # Modify the final fully connected layer
        in_features = self.efficientnet._fc.in_features
        self.additional_fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(64, num_classes)
        )

        # Modify the final fully connected layer of EfficientNet
        self.efficientnet._fc = nn.Identity()

    def forward(self, x):
        # Forward pass through the EfficientNet model
        x = self.efficientnet.extract_features(x)

        # Additional convolutional layers with dropout
        x = self.additional_conv(x)

        # Global average pooling
        x = x.mean([2, 3])

        # Additional fully connected layers with dropout
        x = self.additional_fc(x)

        return x

# Instantiate the modified EfficientNet model with 2 convolutional layers and dropout
num_classes = 8
dropout_rate = 0.2
model = ModifiedEfficientNet(num_classes, dropout_rate=dropout_rate)

# Print the model architecture
print(model)

Loaded pretrained weights for efficientnet-b0
ModifiedEfficientNet(
  (efficientnet): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity(

In [ ]:
import torch

In [ ]:
loaded_model = ModifiedEfficientNet(num_classes, dropout_rate=dropout_rate)

model.load_state_dict(torch.load("/content/drive/MyDrive/save_model/model.pt"))
model.eval()

Loaded pretrained weights for efficientnet-b0


ModifiedEfficientNet(
  (efficientnet): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dSta

In [ ]:
from PIL import Image
import torchvision
from torchvision import transforms

In [ ]:
import torch
from PIL import Image

# Assuming 'model' is your pre-trained model
model.eval()

# Load image
img = Image.open('/content/drive/MyDrive/dataset fix/potato_early_blight/Early_Blight_1.jpg')

# Apply transformation
transform = transforms.ToTensor()
input = transform(img)
input = input.unsqueeze(0)  # Add batch dimension

# Get prediction
with torch.no_grad():
    output = model(input)

# Assuming output is a tensor, you might need to apply softmax or sigmoid based on your model
# For example, if you have a multi-class classification with softmax activation
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Get the predicted class index
predicted_class = torch.argmax(probabilities).item()

class_names = ['Cucumber Healthy', 'Cucumber Unhealthy','Potato Early Blight','Potato Healthy','Potato Late Blight','Tomato Early Blight','Potato Healthy','Potati Late Blight']

# Display the result
print(f"Predicted class index: {predicted_class}")


Predicted class index: 2


In [ ]:
%%writefile app.py
import streamlit as st
from PIL import Image,ImageOps
import random
import numpy as np
from torchvision import transforms
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
import torch

class_names = ['Potato Late Blight', 'Potato Healthy', 'Potato Early Blight', 'Tomato Late Blight', 'Tomato Early Blight', 'Tomato Healthy', 'Cucumber Healty', 'Cucumber Unhealthy']

class ModifiedEfficientNet(nn.Module):
    def __init__(self, num_classes, model_name='efficientnet-b0', dropout_rate=0.2):
        super(ModifiedEfficientNet, self).__init__()

        # Load the pre-trained EfficientNet model
        self.efficientnet = EfficientNet.from_pretrained(model_name)

        # Additional convolutional layer with dropout
        self.additional_conv = nn.Sequential(
            nn.Conv2d(1280, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
        )

        # Modify the final fully connected layer
        in_features = self.efficientnet._fc.in_features
        self.additional_fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(64, num_classes)
        )

        # Modify the final fully connected layer of EfficientNet
        self.efficientnet._fc = nn.Identity()

    def forward(self, x):
        # Forward pass through the EfficientNet model
        x = self.efficientnet.extract_features(x)

        # Additional convolutional layers with dropout
        x = self.additional_conv(x)

        # Global average pooling
        x = x.mean([2, 3])

        # Additional fully connected layers with dropout
        x = self.additional_fc(x)

        return x

# Instantiate the modified EfficientNet model with 2 convolutional layers and dropout
num_classes = 8
dropout_rate = 0.2
model = ModifiedEfficientNet(num_classes, dropout_rate=dropout_rate)

# Print the model architecture
print(model)

loaded_model = ModifiedEfficientNet(num_classes=len(class_names), dropout_rate=dropout_rate)
model.load_state_dict(torch.load("/content/drive/MyDrive/save_model/model.pt"))
model.eval()

# Image preprocessing
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image).unsqueeze(0)
    return image

def predict(image):
    try:
        # Preprocess the image
        input_tensor = preprocess_image(image)

        # Make a prediction
        with torch.no_grad():
            output = model(input_tensor)

        _, predicted_class = torch.max(output, 1)
        class_name = class_names[predicted_class.item()]

        return class_name

    except Exception as e:
        return str(e)

#tampilan sidebar
with st.sidebar :
    st.title('🥬Agrisense')
    uploaded_file = st.file_uploader("Upload gambar", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
            # Tampilkan gambar yang diunggah
            image = Image.open(uploaded_file)
            st.image(image, caption="Gambar yang diunggah", use_column_width=True)
            prediction = predict(image)
            st.success(f"Prediction: {prediction}")


#tampilan utama
if uploaded_file is None : #judul aplikasi hilang ketika diupload gambar
  st.title('🥬Agrisense: Plant Health Monitoring Application based on Multiclass Image Classification to Detect and Diagnose Diseases in Food Plants')
  col1,col2,col3 = st.columns(3)
  with col1 :
    st.caption('')
  with col2 :
    st.caption('Made by Mindwave')
  with col3 :
    st.caption('')


# Tampilkan gambar yang diunggah
col1,col2 = st.columns(2)
with col1 :
  if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Gambar yang diunggah", use_column_width=True)
with col2 :
  if uploaded_file is not None:
    st.header(f"{prediction}")
    if prediction == 'Potato Early Blight' :
      st.write('Penyakit yang disebabkan oleh jamur Alternaria solani yang menyerang tanaman dan biasanya menyebabkan bercak coklat pada daun.')
      st.subheader('Gejala')
      st.write('Penyakit ini meliputi bercak coklat pada daun, bintik-bintik kecil pada buah, dan daun tanaman menjadi kuning dan jatuh.')
    elif prediction == 'Potato Late Blight' :
      st.write('Penyakit yang disebabkan oleh jamur Phytophthora infestans yang menyerang tanaman dan biasanya menyebabkan  berwarna hijau atau coklat gelap yang basah pada daun kentang')
      st.subheader('Gejala')
      st.write('Gejala penyakit ini meliputi bercak-bercak berwarna hijau atau coklat gelap yang basah pada daun kentang, bintik-bintik kecil pada buah, dan batang dan umbi kentang juga dapat terinfeksi')
    elif prediction == 'Potato Healthy' :
      st.write('Tanaman dalam kondisi yang sehat')
    elif prediction == 'Tomato Late Blight' :
      st.write('Late blight disebabkan oleh jamur Phytophthora infestans, dan dapat mengakibatkan kerusakan serius pada daun dan buah tomat.')
      st.subheader('Gejala')
      st.write('Penyakit ini ditandai oleh bercak hitam pada daun dan buah tomat. Bercak ini dapat membesar dan menyebabkan daun dan buah membusuk. Hawar daun dapat menyebar secara cepat dalam kondisi lembab.')
    elif prediction == 'Tomato Early Blight' :
      st.write('Penyakit yang disebabkan oleh jamur Alternaria solani yang menyerang tanaman dan biasanya menyebabkan bercak coklat pada daun dan batang tomat.')
      st.subheader('Gejala')
      st.write('Penyakit ini meliputi bercak coklat pada daun, bintik-bintik kecil pada buah, dan daun tanaman menjadi kuning dan jatuh.')
    elif prediction == 'Tomato Healthy' :
      st.write('Tanaman dalam kondisi yang sehat')
    elif prediction == 'Cucumber Healty' :
      st.write('Tanaman dalam kondisi yang sehat')
    elif prediction == 'Cucumber Unhealthy' :
      st.write('Tanaman mentimun yang kurang air juga menyebabkan daun menguning. Ketika tanaman mentimun menjadi terlalu kering, daunnya akan menggulung, berubah menjadi coklat di bagian tepi, dan memudar dari hijau menjadi kuning, mengering dan rontok.')
      st.subheader('Gejala')
      st.write('mosaik pada daun, klorosis, malformasi daun, vein banding, keriting, dan kuning')
#untuk menampilkan obat obatannya
if uploaded_file is not None :

  if prediction == 'Potato Early Blight' :
    st.subheader('Rekomendasi Obat Tanaman')
    col12,col22,col23 = st.columns(3)
    with col12 :
      image = Image.open('/content/drive/MyDrive/Penyakit kentang/Obat Kentang Early Blight-Fungisida Mancozeb.jpg')
      st.image(image,caption='Mancozeb')
      st.write("Link Pembelian [Mancozeb](https://shopee.co.id/Barozeb-50-gr-Kemasan-Repack-Fungisida-Kontak-Sistemik-Mankozeb-Plus-Silika-Dimetomorf-Mancozeb-Biru-Obat-Pembasmi-Hama-Jamur-Tanaman-Cabe-Buah-Bunga-Sayur-i.135032082.21380267811?sp_atk=495d832a-84aa-4471-8b57-208a8dbbad0f&xptdk=495d832a-84aa-4471-8b57-208a8dbbad0f )")
    with col22 :
      st.write('')
    with col23 :
      st.write('')

  elif prediction == 'Potato Late Blight' :
    st.subheader('Rekomendasi Obat Tanaman')
    col12,col22,col23 = st.columns(3)
    with col12 :
      image = Image.open('/content/drive/MyDrive/Penyakit kentang/Obat Kentang Late Blight-Fungisida Metalaxyl.jpg')
      st.image(image,caption='Metalaxyl')
      st.write("Link Pembelian [Metalaxyl](https://shopee.co.id/BESROMIL-35WP-100-Gram-Fungisida-Sistemik-Metalaxyl-i.9676553.7219962454 )")
    with col22 :
      st.write('')
    with col23 :
      st.write('')
  elif prediction == 'Tomato Late Blight' :
    st.subheader('Rekomendasi Obat Tanaman')
    col12,col22,col32 = st.columns(3)
    with col12 :
      image = Image.open('/content/drive/MyDrive/data /gambar obat /tomato early blight /Mancozeb 80%.jpg')
      st.image(image,caption='Mancozeb 80%')
      st.write("Link Pembelian [Mancozeb 80%](https://shopee.co.id/Chlorothalonil-75-WP-Fungisida-Protektif-Kontak-500-gram-i.59597759.980161323)")
    with col22 :
      image = Image.open('/content/drive/MyDrive/data /gambar obat /tomato late blight/anrin.png')
      st.image(image,caption='Anrin')
      st.write("Link Pembelian [Anrin](https://www.tokopedia.com/kebutuhanpetani/fungisida-anrin-anti-bulai-thiram-80-metalaxyl-25-bakterisida-10-50g?utm_source=google&utm_medium=organic&utm_campaign=pdp-seo)")
    with col32 :
      image = Image.open('/content/drive/MyDrive/data /gambar obat /tomato late blight/Kuprum.jpg')
      st.image(image,caption='Kuprum')
      st.write("Link Pembelian [Kuprum](https://shopee.co.id/CUPROCIDE-77WP-1KG-FUNGISIDA-TEMBAGA-HIDROKSIDA-i.321634711.18944357661)")

  elif prediction == 'Tomato Early Blight' :
    st.subheader('Rekomendasi Obat Tanaman')
    col12,col22= st.columns(2)
    with col12 :
      image = Image.open('/content/drive/MyDrive/data /gambar obat /tomato early blight /Mancozeb 80%.jpg')
      st.image(image,caption='Mancozeb 80%')
      st.write("Link Pembelian [Mancozeb 80%](https://shopee.co.id/Chlorothalonil-75-WP-Fungisida-Protektif-Kontak-500-gram-i.59597759.980161323)")
    with col22 :
      image = Image.open('/content/drive/MyDrive/data /gambar obat /tomato early blight /Chlorothalonil.jpg')
      st.image(image,caption='Chlorothalonil')
      st.write("Link Pembelian [Chlorothalonil](https://shopee.co.id/Chlorothalonil-75-WP-Fungisida-Protektif-Kontak-500-gram-i.59597759.980161323)")

  elif prediction == 'Cucumber Unhealthy' :
    st.subheader('Rekomendasi Obat Tanaman')
    col12,col22 = st.columns(2)
    with col12 :
      image = Image.open('/content/drive/MyDrive/data /gambar obat /cucumber unhealthy/BMW Pupuk Cair.jpg')
      st.image(image,caption='BMW Pupuk Cair')
      st.write("Link Pembelian [BMW Pupuk Cair](https://www.tokopedia.com/lakukerasstoreid/pupuk-organik-cair-penyubur-tanaman-obat-pembasmi-hama-timun?utm_source=google&utm_medium=organic&utm_campaign=pdp-seo)")
    with col22 :
      image = Image.open('/content/drive/MyDrive/data /gambar obat /cucumber unhealthy/Biowasil timun.jpg')
      st.image(image,caption='Biowasil')
      st.write("Link Pembelian [Biowasil](https://www.lazada.co.id/products/langsung-kirim-biowasil-pestisida-pembasmi-hama-timun-paling-ampuh-obat-pengusir-hama-mentimun-bawang-merah-jahe-obat-pembasmi-hama-di-buah-timun-pupuk-semprot-membasmi-serangan-hama-timun-paling-bagus-i6857500007.html)")


Overwriting app.py
